In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'E:\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 200
with open(path + "baseline_200_100_wc", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

260377


In [3]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    return index

In [4]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [8]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list

def unzip(some_list):
    return [ i for i, j in some_list ]

# take nearest 6 and remove the searched word itself
print(unzip(find_nearest_k('desert' ,6)[1:]))#the food or the place
print(unzip(find_nearest_k('address',6)[1:]))#location or to to speak to
print(unzip(find_nearest_k('duck'   ,6)[1:]))#noun (animal) or verb 
print(unzip(find_nearest_k('bear',   6)[1:]))#animal or verb
print(unzip(find_nearest_k('left',   6)[1:]))#adverb (direction) and verb(the plane left)
print(unzip(find_nearest_k('bank',   6)[1:]))# The president of the bank walked along the river bank.
print("--------")

['gobi', 'namib', 'sonoran', 'taklamakan', 'mojave']
['addresses', 'addressing', 'addressed', 'ipv4', 'anycast']
['lame', 'darkwing', 'daffy', 'muscovy', 'whistling']
['grizzly', 'huggy', 'bears', 'teddy', 'spectacled']
['right', 'leaving', 'handers', 'returned', 'after']
['banks', 'idbi', 'grindlays', 'icici', 'barings']
--------


## 3cosAdd 

In [9]:

def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim.numpy(),id_a,id_a_star,id_b)
    return word_dict[index]

# 3cosAdd normalised befor arithmetic

In [10]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim.numpy(),id_a,id_a_star,id_b)
    return word_dict[index]

## 3cos Mult

In [11]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        print(a in id_dict)
        print(a_star in id_dict)
        print(b in id_dict)
        print(b_star in id_dict)
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim.numpy(),id_a,id_a_star,id_b)
    return word_dict[index]

# Test all

In [12]:
def format_results(add,mult):
    add_string = str(add)
    add_string = add_string.lstrip('0')
    add_string = format(add_string, ".4")
    mult_string = str(mult)
    mult_string = mult_string.lstrip('0')
    mult_string = format(mult_string, ".4")
    return add_string +'/'+mult_string


In [15]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
        
        for line_id,line in enumerate(lines):
            #if (line_id % 100 == 0):
            #    print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_syn_sucess[2] += 1
                    
        print(format_results(local_sucesses[1]/float(local_tasks_count),local_sucesses[2]/float(local_tasks_count)))
        print('-------------------')

print('semantical')
print(format_results(count_sem_sucess[1]/float(count_sem_questions),count_sem_sucess[2]/float(count_sem_questions)))

print('syntactical')
print(format_results(count_syn_sucess[1]/float(count_sem_questions),count_syn_sucess[2]/float(count_sem_questions)))

print('overall')
print(format_results((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions),\
                     (count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions)))

capital-common-countries.txt
.952/.946
-------------------
capital-world.txt
.775/.769
-------------------
city-in-state.txt
.532/.533
-------------------
currency.txt
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
Tr

In [16]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [17]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
174824
1460
tf.Tensor(0.7790869118429805, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
174824
174824
tf.Tensor(1.0, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
237142
260365
tf.Tensor(0.7269807913041793, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
225926
260365
tf.Tensor(0.8090076165537934, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
173375
82259
tf.Tensor(0.863230017372904, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
127720
82259
tf.Tensor(0.7881081706862887, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
105268
70044
tf.Tensor(0.8758395461983315, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
188967
37462
tf.Tensor(0.7384337035044879, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
114009
219162
tf.Tensor(0.7640540720731325, shape=(), dtype=float64) 0.7
lineprofessor	doctor	

tf.Tensor(0.7656793740867991, shape=(), dtype=float64) 0.631
linemonk	oracle	5.00

monk oracle
44910
6922
tf.Tensor(0.6359213062147226, shape=(), dtype=float64) 0.5
linecup	food	5.00

cup food
180411
69227
tf.Tensor(0.632790352969132, shape=(), dtype=float64) 0.5
linejournal	association	4.97

journal association
237304
52093
tf.Tensor(0.7466438156359314, shape=(), dtype=float64) 0.497
linestreet	children	4.94

street children
162693
169337
tf.Tensor(0.6988096363728684, shape=(), dtype=float64) 0.49400000000000005
linecar	flight	4.94

car flight
260365
24255
tf.Tensor(0.6951623052609107, shape=(), dtype=float64) 0.49400000000000005
linespace	chemistry	4.88

space chemistry
209979
16373
tf.Tensor(0.6401981454143604, shape=(), dtype=float64) 0.488
linesituation	conclusion	4.81

situation conclusion
62215
212997
tf.Tensor(0.7650853539173189, shape=(), dtype=float64) 0.481
lineword	similarity	4.75

word similarity
3597
223717
tf.Tensor(0.7106341254887136, shape=(), dtype=float64) 0.475
line

tf.Tensor(0.6985377846663237, shape=(), dtype=float64) 0.092
linelad	wizard	0.92

lad wizard
186309
69186
tf.Tensor(0.6924222951583477, shape=(), dtype=float64) 0.092
linesugar	approach	0.88

sugar approach
59245
248049
tf.Tensor(0.5667147454040572, shape=(), dtype=float64) 0.088
linerooster	voyage	0.62

rooster voyage
159257
56156
tf.Tensor(0.5447394980365063, shape=(), dtype=float64) 0.062
linenoon	string	0.54

noon string
149540
82565
tf.Tensor(0.5966651577688284, shape=(), dtype=float64) 0.054000000000000006
linechord	smile	0.54

chord smile
85801
241779
tf.Tensor(0.649905754740504, shape=(), dtype=float64) 0.054000000000000006
lineprofessor	cucumber	0.31

professor cucumber
45172
188967
tf.Tensor(0.555770292254358, shape=(), dtype=float64) 0.031
lineking	cabbage	0.23

king cabbage
61365
18555
tf.Tensor(0.553145167291483, shape=(), dtype=float64) 0.023
Average Distance between prediction and hand assigned is tf.Tensor(0.24136650144720656, shape=(), dtype=float64)


In [18]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.6190559534127181, pvalue=7.278807576052759e-23)